In [44]:
import os
import json
import glob
import psycopg2
import pandas as pd
import datetime
from glom import glom
import re

In [2]:
os.getcwd()

'C:\\Users\\Jaganath Babu\\Desktop\\Data Modelling projects\\Project 1B -song data ETL using cassandra'

In [3]:
os.chdir('C:\\Users\\Jaganath Babu')

In [4]:
os.getcwd()

'C:\\Users\\Jaganath Babu'

In [5]:
confirmed = pd.read_json('covid_19_confirmed.json', lines = False)

In [6]:
vaccined = pd.read_json('covid_19_vaccines.json', lines = False)

In [7]:
with open('covid_19_confirmed.json','r') as f:
    data = json.loads(f.read())
data_keys = data.keys()

In [8]:
len(data_keys)

194

In [9]:
print(data_keys)

dict_keys(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary', 'Iceland', 'India'

In [10]:
def create_dataframe(data_keys, data_df):
    data_df_list_of_dict = []
    for i in data_keys:
        temp_dict = {"country": None, "population": None, "sq_km_area": None, "life_expectancy": None, "elevation_in_meters": None, "continent":None, "abbreviation": None,
                    "location": None, "iso": None, "capital_city": None}
        temp_dict['country'] = str(i)
        if "population" in data_df[i]["All"]:
            temp_dict["population"] = data_df[i]['All']["population"]
        if "sq_km_area" in data_df[i]["All"]:
            temp_dict["sq_km_area"] = data_df[i]['All']["sq_km_area"]
        if "life_expectancy" in data_df[i]["All"]:
            temp_dict["life_expectancy"] = data_df[i]['All']["life_expectancy"]
        if "elevation_in_meters" in data_df[i]["All"]:
            temp_dict["elevation_in_meters"] = data_df[i]['All']["elevation_in_meters"]
        if "continent" in data_df[i]["All"]:
            temp_dict["continent"] = data_df[i]['All']["continent"]
        if "abbreviation" in data_df[i]["All"]:
            temp_dict["abbreviation"] = data_df[i]['All']["abbreviation"]
        if "location" in data_df[i]["All"]:
            temp_dict["location"] = data_df[i]['All']["location"]
        if "iso" in data_df[i]["All"]:
            temp_dict["iso"] = data_df[i]['All']["iso"]
        if "capital_city" in data_df[i]["All"]:
            temp_dict["capital_city"] = data_df[i]['All']["capital_city"]
        data_df_list_of_dict.append(temp_dict)
    return data_df_list_of_dict

In [11]:
new_data = create_dataframe(data_keys, data)

In [12]:
confirmed_df = pd.DataFrame(data = new_data, columns = ["country", "population", "sq_km_area", "life_expectancy", "elevation_in_meters", "continent", "abbreviation",
                    "location", "iso", "capital_city"])

In [13]:
#check for NA and convert them to NONE
confirmed_df.head()

,country,population,sq_km_area,life_expectancy,elevation_in_meters,continent,abbreviation,location,iso,capital_city
0,Afghanistan,35530081.0,652090.0,45.9,None,Asia,AF,Southern and Central Asia,4.0,Kabul
1,Albania,2930187.0,28748.0,71.6,None,Europe,AL,Southern Europe,8.0,Tirana
2,Algeria,41318142.0,2381741.0,69.7,800,Africa,DZ,Northern Africa,12.0,Alger
3,Andorra,76965.0,468.0,83.5,"1,996",Europe,AD,Southern Europe,20.0,Andorra la Vella
4,Angola,29784193.0,1246700.0,38.3,"1,112",Africa,AO,Central Africa,24.0,Luanda


In [82]:
#notice the special characters in country - we need to remove it before doing the ETL
confirmed_df.iloc[30:50]

,country,population,sq_km_area,life_expectancy,elevation_in_meters,continent,abbreviation,location,iso,capital_city
30,Cambodia,1.600537e+07,181035.0,56.5,126,Asia,KH,Southeast Asia,116.0,Phnom Penh
31,Cameroon,2.405373e+07,475442.0,54.8,667,Africa,CM,Central Africa,120.0,Yaound
32,Canada,3.662420e+07,9970610.0,79.4,487,North America,CA,North America,124.0,Ottawa
33,Central African Republic,4.659080e+06,622984.0,44,635,Africa,CF,Central Africa,140.0,Bangui
34,Chad,1.489999e+07,1284000.0,50.5,543,Africa,TD,Central Africa,148.0,N'Djam
35,Chile,1.805473e+07,756626.0,75.7,"1,871",South America,CL,South America,152.0,Santiago de Chile
36,China,1.409517e+09,9572900.0,71.4,"1,840",Asia,CN,Eastern Asia,156.0,Peking
37,Colombia,4.906562e+07,1138914.0,70.3,593,South America,CO,South America,170.0,Santaf
38,Comoros,8.139120e+05,1862.0,60,None,Africa,KM,Eastern Africa,174.0,Moroni
39,Congo (Brazzaville),NaN,NaN,None,None,None,None,None,NaN,None


In [83]:
confirmed_df["country"] = confirmed_df["country"].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [84]:
#special characters are removed
confirmed_df.iloc[30:50]

,country,population,sq_km_area,life_expectancy,elevation_in_meters,continent,abbreviation,location,iso,capital_city
30,Cambodia,1.600537e+07,181035.0,56.5,126,Asia,KH,Southeast Asia,116.0,Phnom Penh
31,Cameroon,2.405373e+07,475442.0,54.8,667,Africa,CM,Central Africa,120.0,Yaound
32,Canada,3.662420e+07,9970610.0,79.4,487,North America,CA,North America,124.0,Ottawa
33,Central African Republic,4.659080e+06,622984.0,44,635,Africa,CF,Central Africa,140.0,Bangui
34,Chad,1.489999e+07,1284000.0,50.5,543,Africa,TD,Central Africa,148.0,N'Djam
35,Chile,1.805473e+07,756626.0,75.7,"1,871",South America,CL,South America,152.0,Santiago de Chile
36,China,1.409517e+09,9572900.0,71.4,"1,840",Asia,CN,Eastern Asia,156.0,Peking
37,Colombia,4.906562e+07,1138914.0,70.3,593,South America,CO,South America,170.0,Santaf
38,Comoros,8.139120e+05,1862.0,60,None,Africa,KM,Eastern Africa,174.0,Moroni
39,Congo Brazzaville,NaN,NaN,None,None,None,None,None,NaN,None


In [16]:
with open('covid_19_vaccines.json','r') as f:
    data_vac = json.loads(f.read())
data_keys_vac = data_vac.keys()

In [17]:
def create_dataframe_vac(data_keys, data_df):
    data_df_list_of_dict = []
    for i in data_keys:
        temp_dict = {'country': None,'administered': None, 'people_vaccinated': None, 'people_partially_vaccinated': None, 'updated': None}
        temp_dict['country'] = str(i)
        if "administered" in data_df[i]["All"]:
            temp_dict["administered"] = data_df[i]['All']["administered"]
        if "people_vaccinated" in data_df[i]["All"]:
            temp_dict["people_vaccinated"] = data_df[i]['All']["people_vaccinated"]
        if "people_partially_vaccinated" in data_df[i]["All"]:
            temp_dict["people_partially_vaccinated"] = data_df[i]['All']["people_partially_vaccinated"]
        if "updated" in data_df[i]["All"]:
            temp_dict["updated"] = data_df[i]['All']["updated"]
        data_df_list_of_dict.append(temp_dict)
    return data_df_list_of_dict

In [18]:
new_data_vac = create_dataframe_vac(data_keys_vac, data_vac)

In [19]:
vaccined_df = pd.DataFrame(data = new_data_vac, columns = ["country", "administered", "people_vaccinated", "people_partially_vaccinated", "updated"])

In [20]:
vaccined_df.head()

,country,administered,people_vaccinated,people_partially_vaccinated,updated
0,Afghanistan,641295,158343,482952,2021/06/10 00:00:00+00
1,Albania,803682,310336,493346,2021/06/10 00:00:00+00
2,Algeria,2500000,0,2500000,2021/06/10 00:00:00+00
3,Andorra,35926,8303,27613,2021/06/10 00:00:00+00
4,Angola,1115919,375630,740289,2021/06/10 00:00:00+00


In [119]:
len(vaccined_df["country"])

165

In [104]:
#replacing special characters in vaccined_df country field for ETL
vaccined_df["country"] = vaccined_df["country"].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [21]:
vaccined_df.loc[vaccined_df['updated'].isnull()]

,country,administered,people_vaccinated,people_partially_vaccinated,updated
164,Global,5227261824,471380933,925965940,None


In [22]:
 def create_dataframe_dates(data_keys, data_df):
    data_df_list_of_list = []
    for i in data_keys:
        date_keys = data_df[i]["All"]["dates"].keys()
        for j in date_keys:
            date_list = []
            date_list.append(i)
            date_list.append(j)
            date_list.append(data_df[i]["All"]["dates"][j])
            data_df_list_of_list.append(date_list)
    return data_df_list_of_list

In [23]:
dates_df_list = create_dataframe_dates(data_keys, data)

In [24]:
dates_df = pd.DataFrame(data = dates_df_list, columns = ["country", "dates", "confirmed_cases"])

In [168]:
#removing special characters from country column in dates_df before ETL
dates_df["country"] = dates_df["country"].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [26]:
con = psycopg2.connect("user=postgres password='@Jgun220'");
con.set_session(autocommit=True)

In [27]:
cursor = con.cursor();

In [ ]:
name_Database   = "Jagan";
sqlCreateDatabase = "create database "+name_Database+";"
cursor.execute(sqlCreateDatabase);

In [28]:
con.close()    

In [29]:
con = psycopg2.connect("host=127.0.0.1 dbname=jagan user=postgres password='@Jgun220'");
con.set_session(autocommit=True)

In [30]:
cursor = con.cursor()

In [162]:
country_table_create = ("""CREATE TABLE IF NOT EXISTS country_info (country_id int PRIMARY KEY, country varchar, population numeric null, sq_km_area numeric null, life_expectancy numeric null, elevation_in_meters varchar null,
continent char(100) null, abbreviation char(2) null, location char(100) null, iso numeric null, capital_city char(50) null);""")

In [163]:
vaccine_table_create = ("""CREATE TABLE IF NOT EXISTS vaccined_info (vaccine_id int PRIMARY KEY , country_id int, administered bigint null, people_vaccinated int null, people_partially_vaccinated int null, updated timestamp null, 
CONSTRAINT fk_country FOREIGN KEY(country_id) REFERENCES country_info(country_id));""")

In [164]:
confirmed_cases_table_create = ("""CREATE TABLE IF NOT EXISTS confirmed_cases (case_id int PRIMARY KEY, country_id int, date date null, confirmed_cases int null, CONSTRAINT fk_confirm_country
FOREIGN KEY(country_id) REFERENCES country_info(country_id));""")

In [165]:
cursor.execute(country_table_create)

In [166]:
cursor.execute(vaccine_table_create)

In [167]:
cursor.execute(confirmed_cases_table_create)

In [37]:
country_data = confirmed_df.values[0].tolist()

In [149]:
#songplay_table_insert = ("INSERT INTO songplays (songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)")
country_table_insert = ("""INSERT INTO country_info (country_id, country, population, sq_km_area, life_expectancy, elevation_in_meters,
continent, abbreviation, location, iso, capital_city) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, %s) """)

In [150]:
vaccine_table_insert = ("""INSERT INTO vaccined_info (vaccine_id, country_id, administered, people_vaccinated, people_partially_vaccinated, updated) VALUES (%s,%s,%s,%s,%s,%s) """)

In [151]:
vaccine_country_select = ("""select country_id from country_info where country_info.country = '{0}'""") 

In [152]:
confirmed_table_insert = ("""INSERT INTO confirmed_cases (case_id, country_id, date, confirmed_cases) VALUES (%s,%s,%s,%s)""")

In [153]:
dates_country_select = ("""select country_id from country_info where country_info.country = '{0}'""")

In [133]:
dates_df.head()

,country,dates,confirmed_cases
0,Afghanistan,2021-06-09,84050
1,Afghanistan,2021-06-08,82326
2,Afghanistan,2021-06-07,80841
3,Afghanistan,2021-06-06,79224
4,Afghanistan,2021-06-05,77963


In [172]:
print("{0} rows inserted in country_info_tables".format(len(confirmed_df)))

194 rows inserted in country_info_tables


In [73]:
country_etl_data = (0, country_data[0], country_data[1], country_data[2], country_data[3], country_data[4], country_data[5], country_data[6], country_data[7], country_data[8], country_data[9])

In [74]:
cursor.execute(country_table_insert, country_etl_data)

In [ ]:
cursor.execute(("""DROP TABLE IF EXISTS country_info CASCADE"""))
cursor.execute(("""DROP TABLE IF EXISTS vaccined_info"""))
cursor.execute(("""DROP TABLE IF EXISTS confirmed_cases"""))
cursor.execute(("""delete from country_info"""))
cursor.execute(("""delete from vaccined_info"""))

In [169]:
#country_info table ETL
for i, row in confirmed_df.iterrows():
    cursor.execute(country_table_insert, (i, row[0], row[1], row[2], row[3], str(row[4]), row[5], row[6], row[7], row[8], row[9]))

In [190]:
vaccined_data = vaccined_df.values[0].tolist()

In [158]:
vaccined_df.head(5)

,country,administered,people_vaccinated,people_partially_vaccinated,updated
0,Afghanistan,641295,158343,482952,2021/06/10 00:00:00+00
1,Albania,803682,310336,493346,2021/06/10 00:00:00+00
2,Algeria,2500000,0,2500000,2021/06/10 00:00:00+00
3,Andorra,35926,8303,27613,2021/06/10 00:00:00+00
4,Angola,1115919,375630,740289,2021/06/10 00:00:00+00


In [170]:
#vaccined_info table ETL
for i, row in vaccined_df.iterrows():
    cursor.execute(vaccine_country_select.format((row[0])))
    results = cursor.fetchone()
    
    if results:
        country_id = results[0]
    else:
        country_id = None
    
    cursor.execute(vaccine_table_insert, (i, country_id, row[1], row[2], row[3], row[4]))

In [132]:
#vaccined_df.head()
vaccined_df.loc[vaccined_df["updated"].isnull()]

,country,administered,people_vaccinated,people_partially_vaccinated,updated
164,Global,5227261824,471380933,925965940,None


In [47]:
df_nested_list = pd.json_normalize(
    data, 
    record_path = ['All'])

#####
df = pd.read_json('data/nested_deep.json')
df['students'].apply(lambda row: glom(row, 'grade.math'))
####new_data = pd.read_json('covid_19_confirmed.json', orient= 'columns')

KeyError: 'All'

In [ ]:
def create_dataframe(data_keys, data_df):
    data_df_list_of_dict = []
    for i in data_keys:
        temp_dict = {}
        data_df_list_of_dict.append({"country": str(i), "population": data_df[i]['All']["population"] , 
                                     "sq_km_area": data_df[i]['All']["sq_km_area"],
                                    "life_expectancy": data_df[i]['All']["life_expectancy"], "elevation_in_meters": data_df[i]['All']["elevation_in_meters"],
                                    "continent": data_df[i]['All']["continent"],"abbreviation": data_df[i]['All']["abbreviation"], "location": data_df[i]['All']["location"],
                                    "iso": data_df[i]['All']["iso"], "capital_city": data_df[i]['All']["capital_city"]})
    return data_df_list_of_dict   

new_data = create_dataframe(data_keys, data)

In [48]:
new  = data['Afghanistan']['All']["population"] if 'population' in data['Afghanistan']['All'] else 'population': None
data_df[i]['All']["population"] if 'population' in data_df[i]['All'] else 'population':'null'

SyntaxError: invalid syntax (<ipython-input-48-8d2b4a9129ed>, line 1)

In [50]:
##def create_dataframe(data_keys, data_df):
    data_df_list_of_dict = []
    for i in data_keys:
        temp_dict = {}
        temp_dict['country'] = str(i), temp_dict['population'] = data[i]['All']["population"] if 'population' in data[i]['All'] else temp_dict['population'] = None,
            "population":data_df[i]['All']["population"] if 'population' in data_df[i]['All'] else 'population': None, 
                                     "sq_km_area": data_df[i]['All']["sq_km_area"],
                                    "life_expectancy": data_df[i]['All']["life_expectancy"], "elevation_in_meters": data_df[i]['All']["elevation_in_meters"],
                                    "continent": data_df[i]['All']["continent"],"abbreviation": data_df[i]['All']["abbreviation"], "location": data_df[i]['All']["location"],
                                    "iso": data_df[i]['All']["iso"], "capital_city": data_df[i]['All']["capital_city"]})
    return data_df_list_of_dict 


{temp_dict['population'] : data[i]['All']["population"] if 'population' in data[i]['All'] else temp_dict['population'] = None}

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [ ]:
##def create_dataframe(data_keys, data_df):
    data_df_list_of_dict = []
    for i in data_keys:
        temp_dict = {}
        data_df_list_of_dict.append({"country": str(i), "population": data_df[i]['All']["population"] , 
                                     "sq_km_area": data_df[i]['All']["sq_km_area"],
                                    "life_expectancy": data_df[i]['All']["life_expectancy"], "elevation_in_meters": data_df[i]['All']["elevation_in_meters"],
                                    "continent": data_df[i]['All']["continent"],"abbreviation": data_df[i]['All']["abbreviation"], "location": data_df[i]['All']["location"],
                                    "iso": data_df[i]['All']["iso"], "capital_city": data_df[i]['All']["capital_city"]})
    return data_df_list_of_dict